# Interviewer Occupation Detection 

Authors: Ahmed Z. , Cohen H. and Spence S.

# Introduction
(overview of approach)


# Related Work





# Data Pre-Processing Methods

We developed four filters that can be applied to the data. During development of our classification algorithms, we tested on all 16 combinations of those four filters being on or off. For example, the ones and zeroes in the file named "train-0111.csv" correspond to whether each filter is on or off. For "0111":

0 remove_stops. Set to false, do not remove stops words like "at" and "the".
1 stem. Set to true, stem words like "running" into "run". Used Python's NLTK SnowballStemmer.
1 remove_tokens. Set to true - remove all tokens (comma, dash, question mark). Does not remove periods though.
1 remove_periods. Set to true - remove all periods.

We also applied two more filters regardless of filter settings. All characters outside a (generous) whitelist were removed. Finally, all words were set to lowercase, except for unusual capitalization, like capitals not succeeding a period, or many capitals like in acronyms.

Tokens that were not removed were instead replaced by "_comma", "_period", etc. The script "cleaner.py" was used to produce the 16 CSV files.

# Feature Design and Selection Methods

have we designed any features? 
How about is_interviewer or is_interviewee


One major feature in most of our algorithms was the extensive use of ngrams. The Gimli and Pippin approaches especially, when examining text, extract from the text all permutations of 1-grams, 2-grams, 3-grams, etc, up to some limit (usually  4). The 


In [2]:
from stuart.utilities import *
words=("I","am","a","sentence","_period")
print(get_ngrams(words,3))
print(get_cumulative_ngrams(words,3))

ImportError: No module named 'utilities'

# Algorithm Selection

## Baseline

Picked a Bernoulli Naive Bayes and made it multiclass using the 1 vs all approach. We also used indicator bag of words rather than a frequency bag of words, this is because the exerpts were short enough that the words will not appear often enough to be counted highly 
(i have analysis of this.... will put that in here....)

Maybe I can also find words that appear with frequency 2 multiple times and add that in as another binary feature?

## Standard

Stuarts speil on decision trees?

## Advanced

We need to get on with this ASAP

# Optimization 

- if any used

# Parameter selection

- here I can probably do an analysis of which gamma works best.

# Testing + Validation 

- done outside of Kaggle

(mention how first two filters raised accuracy generally, while second two didn't do much -Stuart)

# Pros and Cons of approach and methodology

We	hereby	state	that	all	the	work presented	in this	report	is	that	of	the	authors

Statement of Contributions

- Zafarali Ahmed
- Stuart Spence
- Hannah Cohen